In [8]:
pwd

'/content'

In [4]:
# Load Larger LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import os
# load ascii text and covert to lowercase

full_text = []

for filename in os.listdir(os.getcwd()):
    if (filename!='sample_data' and filename!='.config'):
        raw_text = open(filename, 'r', encoding='utf-8').read()
        full_text.append(raw_text.lower())
flat_list = (', '.join(full_text))
print(len(flat_list))
flat_list = flat_list[:100000].rstrip()

644636


In [0]:
whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ.?!')
answer = ''.join(filter(whitelist.__contains__, flat_list))


In [0]:
# create mapping of unique chars to integers
chars = sorted(list(set(answer)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [7]:
print(char_to_int)

{' ': 0, '!': 1, '.': 2, '?': 3, 'a': 4, 'b': 5, 'c': 6, 'd': 7, 'e': 8, 'f': 9, 'g': 10, 'h': 11, 'i': 12, 'j': 13, 'k': 14, 'l': 15, 'm': 16, 'n': 17, 'o': 18, 'p': 19, 'q': 20, 'r': 21, 's': 22, 't': 23, 'u': 24, 'v': 25, 'w': 26, 'x': 27, 'y': 28, 'z': 29}


In [8]:
# summarize the loaded data
n_chars = len(answer)
n_vocab = len(chars)
print( "Total Characters: ", n_chars)
print( "Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = answer[i:i + seq_length]
	seq_out = answer[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="../weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model

Total Characters:  96210
Total Vocab:  30
Total Patterns:  96110




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [9]:
model.fit(X, y, epochs=40, batch_size=128, callbacks=callbacks_list)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/40





96110/96110 [==============================] - 308s 3ms/step - loss: 2.8794

Epoch 00001: loss improved from inf to 2.87937, saving model to ../weights-improvement-01-2.8794-bigger.hdf5
Epoch 2/40
96110/96110 [==============================] - 302s 3ms/step - loss: 2.6553

Epoch 00002: loss improved from 2.87937 to 2.65534, saving model to ../weights-improvement-02-2.6553-bigger.hdf5
Epoch 3/40
96110/96110 [==============================] - 302s 3ms/step - loss: 2.5083

Epoch 00003: loss improved from 2.65534 to 2.50833, saving model to ../weights-improvement-03-2.5083-bigger.hdf5
Epoch 4/40
96110/96110 [==============================] - 305s 3ms/step - loss: 2.3926

Epoch 00004: loss improved from 2.50833 to 2.39258, saving model to ../weights-improvement-04-2.3926-bigger.hdf5
Epoch 5/40
96110/96110 [==============================] - 303s 3ms/step - loss: 2.2983

Epoch 0000

In [12]:
int_to_char = dict((i, c) for i, c in enumerate(chars))
...
for i in range(10): 
    # pick a random seed
    #start = numpy.random.randint(0, len(dataX)-1)
    pattern = dataX[i*100-1]
    print ("Seed:")
    print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
    # generate characters
    for i in range(100):
        x = numpy.reshape(pattern, (1, len(pattern), 1))
        x = x / float(n_vocab)
        prediction = model.predict(x, verbose=0)
        index = numpy.argmax(prediction)
        result = int_to_char[index]
        seq_in = [int_to_char[value] for value in pattern]
        sys.stdout.write(result)
        pattern.append(index)
        pattern = pattern[1:len(pattern)]
    print ("\nDone.")

Seed:
" used to shoot reagan. but not anymore. remember? listen stan. i know you love reagan. but you screwe "
d it for a good care.  she couser on scerol be a seconl  the move is someshing in the past cut i hav
Done.
Seed:
" . here at school you call me coach dipwad! where did those dogs come from? you ditching gym class to "
 iild of to a boob.  you gave this carver to sell me she starte.  stan smiths mnd for my cupband soa
Done.
Seed:
" o? today is track. i hate running. then were on the same page. welcome to my sanctuary. look at them "
 a puer to iilljal first siing to the stattes wer wo thin. i have a good cack to the stantaliy grom 
Done.
Seed:
"  scurrying around like ants. go ahead exercise all you want youll never escape the smoky death of ti "
me. i have a teally bean of chance to me.  what are you doing to the mast couerser tiing on the stre
Done.
Seed:
" mes magnifying glass. wow thats a little dark. not at all. life is a banquet.  and death is dessert. "
  i cont know if 